# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict
import functools

In [2]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.memory = OrderedDict()

    def __getitem__(self, key):
        if key not in self.memory:
            return None
        self.memory.move_to_end(key)
        return self.memory[key]

    def __setitem__(self, key, value):
        self.memory[key] = value
        if len(self.memory) > self.max_count:
            self.memory.popitem(last=False)

#### Реализуйте декоратор

In [3]:
def cached(max_count):
    def decorator(func):
        cache = LruCache(max_count)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            if cache[key]:
                print("From memory: key =", key, "result =", cache[key])
                return cache[key]
            result = func(*args, **kwargs)
            cache[key] = result
            return result
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [4]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(3)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

*Примеры работы.*

In [5]:
print("Общий пример")
print(fact(20))
print(fact(5))
print(fact(10))
print(fact(21))
print("Пример того, что хранится ровно max_count элементов")
print(fact1(1))
print(fact1(2))
print(fact1(3))
print(fact1(4))
print(fact1(1)) # Значание для этого аргумента уже забылось

Общий пример
2432902008176640000
From memory: key = ((5,), frozenset()) result = 120
120
From memory: key = ((10,), frozenset()) result = 3628800
3628800
From memory: key = ((20,), frozenset()) result = 2432902008176640000
51090942171709440000
Пример того, что хранится ровно max_count элементов
1
From memory: key = ((1,), frozenset()) result = 1
2
From memory: key = ((2,), frozenset()) result = 2
6
From memory: key = ((3,), frozenset()) result = 6
24
1


#### Сравните свою реализацию с lru_cache из functools

In [6]:
def cached(max_count):
    def decorator(func):
        cache = LruCache(max_count)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            if cache[key]:
                return cache[key]
            result = func(*args, **kwargs)
            cache[key] = result
            return result
        return wrapper
    return decorator

In [7]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@functools.lru_cache(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [8]:
%%time
print(fact(10))
print(fact(20))
print(fact(30))
print(fact(50))

3628800
2432902008176640000
265252859812191058636308480000000
30414093201713378043612608166064768844377641568960512000000000000
CPU times: user 1.4 ms, sys: 0 ns, total: 1.4 ms
Wall time: 1.47 ms


In [9]:
%%time
print(fact1(10))
print(fact1(20))
print(fact1(30))
print(fact1(50))

3628800
2432902008176640000
265252859812191058636308480000000
30414093201713378043612608166064768844377641568960512000000000000
CPU times: user 1.81 ms, sys: 15 µs, total: 1.83 ms
Wall time: 1.67 ms


*Вывод.* Реализация из коробки отрабатывает примерно за то же время.

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [10]:
# <your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [13]:
def checked(*types):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args):
            for arg, kind in zip(args, types):
                if type(arg) != kind:
                    raise TypeError
            return func(*args)
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [14]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [19]:
strange_func("hello", 5, [])

None


## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [ ]:
class Logger:
    def __init__(self, func)